In [1]:
import torch
import math
import numpy as np
import scipy.stats
from scipy.stats import rankdata
from scipy.interpolate import UnivariateSpline

In [2]:
def simulatingU(x): 
    """
    Given an vector x this will return a single U vector of size 1xM where i assume M is the number of observations 
    x(1xM)/np.array: Represents one functional data point. Where M is the number of observations. 
    """
    m = x.shape[1]
    sigma = torch.zeros(m,m,dtype=torch.float64)
    sigma = torch.zeros(m,m)
    for i in range(0,m): 
        newRow = list()
        for j in range(i,x.shape[1]):
            value = math.e**(-5* abs(x[0][i]-x[0][j]))
            sigma[i][j]=value
    sigma = (sigma + sigma.T).fill_diagonal_(1)
    z = torch.normal(mean=0, std=1, size=(m,1)) # m by 1 vector
    y = torch.matmul(torch.sqrt(sigma),z) 
    u = y/torch.norm(y)
    return u.T

In [28]:
# where y is a matrix of J x N 
# where z is a vector of J x 1 
# returns a J x 1 vector 
def compute_f_Hat(y,z): 
    """
    This function checks each row in y to check if each element is less than z in that same row
    Parameters: 
    y (JxN Matrix): Where J represents the number of unit Vectors and N represents the number of functions 
    z (Jx1): A vector to compare each row by. 
    """
    count_less_than_threshold = (y <= z[:, np.newaxis]).sum(axis=1)
    result = count_less_than_threshold/ y.shape[1]
    return result


# arr represents a singular function in points on a grid 1 x M ( where M represents the number of points n the grid)
# Xn represents the matrix for all functions N x M (Where N is the number of functions)
# U represents the matrix of random numbers pulled from the normal distrubution with shape J x M 
# Where J represents the unit vectors and M represents the number of points
# This function will return the depth of function x in respect to all other functions in the data set

def depth(arr, u, xn_uj):
    """
    The depth function will tell us the depth of any given functional data point relative to the data set 
    Parameters: 
    arr(1xM)/np.array: Represents one functional data point. Where M is the number of observations. 
    xn(NxM)np.array: Represents all functional data points. 
    u(JxM)np.array: Represents the unit vectors, where J is the number of unit vectors and M is number of points.
    Returns: 
    np.array : An double that represents the depth of each function relative to eachother. 
    """
    arr_u_dot = torch.matmul(u,arr)
    fHatVector = compute_f_Hat(xn_uj,arr_u_dot)
    result = ((torch.matmul(fHatVector, 1- fHatVector))/u.shape[0])
    return result


def getDepth(functionalDataSet,UnitVectorMatrix):
    xn = torch.from_numpy(functionalDataSet)
    uj = torch.from_numpy(UnitVectorMatrix)
    xn_uj = torch.matmul(xn,uj.T).T # jxn 
    depthList = list()
    for row in xn:
        depthList.append(depth(row,uj,xn_uj).item())    
    # Have to convert to a numpy array since scipcy doesnt like Tensors
    depthVector = np.array(depthList)
    return depthVector



# We might need to add a flag for how rank method is caluclated 
def KW_H_Test(functionalDataSet: np.ndarray, UnitVectorMatrix: np.ndarray, 
              groups:np.ndarray): 
    
    # # Gets the rank giving us a Nx1 vector 
    x = getDepth(functionalDataSet,UnitVectorMatrix)
    rankVector = rankdata(x)
        
    # there is a scipy version of kruskal wallis we can use 
    groupRankDict = dict()
    for i in range(0,groups.shape[0]): # Runs at O(N)
        oldValue = groupRankDict.get(groups[i])
        if oldValue != None: # the key value exsits 
            groupRankDict[groups[i]] = oldValue + rankVector[i]
        else:
            groupRankDict[groups[i]] = rankVector[i]

    unique, counts = np.unique(groups, return_counts=True) # Runs at O(Groups) which should be fast 
    
    summation = 0 
    for key, value in dict(zip(unique, counts)).items(): # Runs at O(N)
        summation += (groupRankDict[key] **2) / value
    
    hStat =  12 /(groups.size* (groups.size +1)) *summation - (3*(groups.size +1))
    degreeOfFreedom = unique.size -1 
    chiSquare = scipy.stats.chi2.ppf(1-.05, df=degreeOfFreedom)
    return(hStat>chiSquare)

Xn = np.random.random((100, 10)) # represents all the data  in points
U = np.random.random((20, 10))   # unit matrix
groups = np.random.randint(1, 11, size=100) # where size must be the amount of rows in Xn 
temp = KW_H_Test(Xn,U,groups)


[0.215305   0.173595   0.21542001 0.15436499 0.08618    0.103475
 0.21224001 0.22098002 0.078135   0.17938    0.21241002 0.22144
 0.18651001 0.122605   0.22725502 0.20598002 0.09166    0.062325
 0.18627501 0.197355   0.10418    0.04928499 0.21845503 0.11091
 0.18500499 0.16417    0.079855   0.19238001 0.18506999 0.20183501
 0.17128    0.207835   0.14778    0.13762499 0.17133    0.18593499
 0.21293998 0.22160001 0.225475   0.02998    0.199705   0.089825
 0.057835   0.13210499 0.195585   0.129035   0.21267    0.21346501
 0.21575499 0.07242    0.18009    0.200105   0.149575   0.19549
 0.11429501 0.21700498 0.17249501 0.193275   0.16446    0.11215
 0.18436    0.18930499 0.22411498 0.13701999 0.09181999 0.18807499
 0.18622501 0.21191001 0.20222501 0.12771001 0.02023    0.030715
 0.18185    0.21461999 0.227845   0.22247501 0.19196501 0.1732
 0.16831002 0.22247    0.038025   0.16943499 0.22666001 0.22350001
 0.234835   0.19254999 0.12943001 0.196445   0.16989    0.200165
 0.21637002 0.220595 

In [25]:
# Gets the derivate of a single function resented by points
# Is able to find derivatives by using interpolation of the points
# To calculate the dirivative of a dataset Xn we need to know at which t they are at, so for now we will make the assumption that they are equally seperated at 1 unit apart 
def getDirivativeOfRow(x,y):
    spline = UnivariateSpline(x, y, s=0)  
    derivative = spline.derivative()
    derivative_values = derivative(x)
    return torch.tensor(derivative_values)


def simplicial_Depth():
    return 0

# X is a row vector with M number of points
def depth2 (x,U, xn,yn):
    """
    X(1xM)/np.array: Represents one functional data point. Where M is the number of observations. 
    U(JxM)np.array: Represents the unit vectors, where J is the number of unit vectors and M is number of points.
    returns the depth of a single row of Xn averaged out
    """
    xPrime = getDirivativeOfRow(T,x) # 1 x M
    # For every Unit vector in Matrix U
    average = 0 
    for j in range(0, U.shape[0]): # 1xM
        xUj = torch.matmul(torch.tensor(x),torch.tensor(U[j]))
        xPrimeUj = torch.matmul(xPrime,torch.tensor(U[j]))
        average += simplicial_Depth(xUj, xPrimeUj, xn_uj, yn_uj)

    return average/U.shape[0]
def getDepth2(functionalDataSet,UnitVectorMatrix):
    # Convert to Tensors
    xn = torch.from_numpy(functionalDataSet)
    uj = torch.from_numpy(UnitVectorMatrix)

    T = torch.arange(xn.shape[1]) # create some T that represents at what time the observations were made

    yn = torch.stack([getDirivativeOfRow(T,row) for row in Xn]) # Take the derivative of functional data set
    
    xn_uj = torch.matmul(xn,uj.T).T # jxn 
    yn_uj = torch.matmul(yn,uj.T).T # jxn 
    
    # Returns a N x 1 vector representing the depth for each row of functionalDataset
    depthList = list()
    for row in xn:
        rowPrime = getDirivativeOfRow(T,row)
        
        depthList.append(depth2(row,uj,xn_uj).item())    
    # Have to convert to a numpy array since scipcy doesnt like Tensors
    depthVector = np.array(depthList)
    return depthVector


Xn = np.random.random((100, 10)) # represents all the data  in points
U = np.random.random((20, 10))   # unit matrix
temp = getDepth2(Xn,U)


# T = torch.arange(Xn.shape[1]) 
# result_row_wise = torch.stack([getDirivativeOfRow(T,row) for row in Xn])

# To get the dirivate app

TypeError: depth2() missing 1 required positional argument: 'yn'

In [ ]:
Xn = torch.tensor(np.random.random((100, 10)))

In [ ]:
# likelyhood depth instead of Simplicial depth

How to compute multiple row operations at the same time: https://saturncloud.io/blog/pytorch-batch-rowwise-application-of-function/#:~:text=PyTorch%20provides%20an%20efficient%20way%20to%20apply%20a%20function%20row,that%20applies%20to%20each%20row.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
rng = np.random.default_rng()
x = np.linspace(-3, 3, 50)
y = np.exp(-x**2) + 0.1 * rng.standard_normal(50)
plt.plot(x, y, 'ro', ms=5)

In [ ]:
spl = UnivariateSpline(x, y)
xs = np.linspace(-3, 3, 100)
plt.plot(xs, spl(xs), 'g', lw=3)

In [ ]:
spl.set_smoothing_factor(0.5)
plt.plot(xs, spl(xs), 'b', lw=3)
plt.show()